In [ ]:
from os.path import join, dirname, basename
from csv import reader as csv_reader
from csv import writer as csv_writer
import datetime
datetime_strptime = datetime.datetime.strptime

In [3]:
def main():
    """メイン関数"""

    # 分割情報から分割日と分割割合を取得
    #split_file = r'***\split_datas.txt'
    split_file = '9984.txt'
    split_datas = get_split_datas(split_file)

    # 係数を計算
    coefficients = calc_coefficients(split_datas)

    # 株価を取得
    #price_file = r'***\price.csv'
    price_file = '9984-ソフトバンクグループ.csv'  
    price_datas = get_price_datas(price_file)

    # 株価に係数を乗じて株価調整
    adj_price_datas = adjust_prices(price_datas, coefficients)


    # 調整した株価を保存
    adj_price_file = join(
        dirname(price_file), 'adj_%s' % basename(price_file),
        )
    with open(
        adj_price_file, 'w', encoding='utf-8-sig', newline='',
        ) as f:
        w = csv_writer(f)
        w.writerows(adj_price_datas)
    return

In [4]:
def get_split_datas(file):
    """分割情報から分割日と分割割合を取得"""
    datas = []

    with open(file, mode='r', encoding='utf-8-sig') as f:
        for line in f:
            # データ例
            # '[1:100]（19/01/12）、[1:2]（19/01/15）、[10:1]（19/01/18）'

            # 読点で分割
            for t in line.split('、'):

                # 末尾の改行を削除
                t = t.rstrip('\r\n')

                # 分割情報と分割日を分ける
                t = t.split(']（')

                # 分割情報から数値を取得
                t_split = t[0].strip('[')
                t_split = t_split.split(':')
                t_split = [float(x) for x in t_split]

                # 分割日を取得
                t_date = datetime_strptime(t[1], '%y/%m/%d）')

                # リストに追加
                # datas: [0]分割日 [1]分割前 [2]分割後
                datas.append([t_date, t_split[0], t_split[1]])

    # リストを分割日の降順でソート
    datas.sort(key=lambda x:x[0], reverse=True)

    return datas

In [5]:
def calc_coefficients(datas):
    """分割情報から株価に乗じる係数を計算する"""

    ######################################
    # (1) 最後の分割日から、西暦9999年12月31日の直前までの期間
    # (2) 前回の分割日から、最後に分割される日の直前までの期間
    # (3) 西暦0001年01月01日から、最初に分割される日の直前までの期間
    ######################################
    # 初期設定

    labels = ['分割日', '次回の分割日', '係数', '分割前', '分割後']
    coefficients = []

    # 9999-12-31 23:59:59.999999
    datetime_max = datetime.datetime.max

    # 0001-01-01 00:00:00
    datetime_min = datetime.datetime.min

    # 一時変数
    t_split_date = None # 分割日
    t_coefficient = 1.0 # 係数
    t_befor = None # 分割前
    t_after = None # 分割後


    ######################################
    # (1) 最後の分割日から、西暦9999年12月31日の直前までの期間

    #    分割日      分割前  分割後
    (split_date, befor, after) = datas[0]

    coefficients.append([
        # 分割日     次回の分割日   係数           分割前    分割後
        split_date, datetime_max, t_coefficient, t_befor, t_after,
        ])

    # 一時変数を更新
    t_split_date = split_date
    t_coefficient = (befor / after)
    t_befor = befor
    t_after = after


    ######################################
    # (2) 前回の分割日から、最後に分割される日の直前までの期間
    # ...

    for (split_date, befor, after) in datas[1:]:
        coefficients.append([
            split_date, t_split_date, t_coefficient, t_befor, t_after,
            ])

        # 一時変数を更新
        t_split_date = split_date
        t_coefficient = t_coefficient * (befor / after)
        t_befor = befor
        t_after = after


    ######################################
    # (3) 西暦0001年01月01日から、最初に分割される日の直前までの期間

    coefficients.append([
        datetime_min, t_split_date, t_coefficient, t_befor, t_after,
        ])

    return (labels, coefficients)

In [6]:
def get_price_datas(file):
    """株価を取得"""

    with open(file, 'r', encoding='utf-8-sig', newline='') as f:
        # [0]日付 [1]始値 [2]高値 [3]安値 [4]終値

        # ラベル行を取得
        labels = next(csv_reader(f))

        # データ行を取得(型変換も行う)
        datas = []
        for c in csv_reader(f):
            datas.append([
                datetime_strptime(c[0], '%Y/%m/%d'),
                float(c[1].replace(',', '')) if c[1] else None,
                float(c[2].replace(',', '')) if c[2] else None,
                float(c[3].replace(',', '')) if c[3] else None,
                float(c[4].replace(',', '')) if c[4] else None,
                ])

    # 日付の昇順でソート
    datas.sort(key=lambda x:x[0], reverse=False)

    return (labels, datas)

In [7]:
def adjust_prices(price_datas, coefficients):
    """株価に係数を乗じて株価調整"""

    # ラベル行を分離
    price_labels = price_datas[0]
    prices_datas = price_datas[1]

    coefficient_labels = coefficients[0]
    coefficients = coefficients[1]

    # ラベル行を生成
    # ['日付', '始値', '高値', '安値', '終値'] + ['係数']
    labels = price_labels + coefficient_labels[2:3]


    datas = []
    datas.append(labels)

    # 株価リスト(日付の昇順)
    for (日付, 始値, 高値, 安値, 終値) in prices_datas:

        # 係数リスト(分割日の降順)
        for (分割日, 次回の分割日, 係数, 分割前, 分割後) in coefficients:
            if 日付 >= 分割日:
                # 分割日 以降の日付でbreak
                break

        # 株価に係数を乗じる
        datas.append([
            日付,
            None if 始値 is None else 始値 * 係数,
            None if 高値 is None else 高値 * 係数,
            None if 安値 is None else 安値 * 係数,
            None if 終値 is None else 終値 * 係数,
            係数,
            ])

    return datas

In [ ]:
#end